In [ ]:
!pip install

ERROR: You must give at least one requirement to install (see "pip help install")


In [ ]:
!pip install datasets
!pip install seqeval

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 9.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential,layers
from tensorflow.keras.layers import Dense,Bidirectional,LSTM,TimeDistributed,SimpleRNN,GRU
import seqeval
from datasets import load_dataset
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
dataset=load_dataset('conll2003')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/12.3k [00:00<?, ?B/s]

conll2003.py:   0%|          | 0.00/9.57k [00:00<?, ?B/s]

The repository for conll2003 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/conll2003.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3453 [00:00<?, ? examples/s]

In [ ]:
training=dataset['train']
validation=dataset['validation']
test=dataset['test']
print(training[5])

{'id': '5', 'tokens': ['"', 'We', 'do', "n't", 'support', 'any', 'such', 'recommendation', 'because', 'we', 'do', "n't", 'see', 'any', 'grounds', 'for', 'it', ',', '"', 'the', 'Commission', "'s", 'chief', 'spokesman', 'Nikolaus', 'van', 'der', 'Pas', 'told', 'a', 'news', 'briefing', '.'], 'pos_tags': [0, 28, 41, 30, 37, 12, 16, 21, 15, 28, 41, 30, 37, 12, 24, 15, 28, 6, 0, 12, 22, 27, 16, 21, 22, 22, 14, 22, 38, 12, 21, 21, 7], 'chunk_tags': [0, 11, 21, 22, 22, 11, 12, 12, 17, 11, 21, 22, 22, 11, 12, 13, 11, 0, 0, 11, 12, 11, 12, 12, 12, 12, 12, 12, 21, 11, 12, 12, 0], 'ner_tags': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 1, 2, 2, 2, 0, 0, 0, 0, 0]}


In [ ]:
training_tokens=training['tokens']
training_tags=training['ner_tags']

validation_tokens=validation['tokens']
validation_tags=validation['ner_tags']

test_tokens=test['tokens']
test_tags=test['ner_tags']
print(training_tags)
print(training_tokens)

[[3, 0, 7, 0, 0, 0, 7, 0, 0], [1, 2], [5, 0], [0, 3, 4, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [5, 0, 0, 0, 0, 3, 4, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 1, 2, 2, 2, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 4, 0], [0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 7, 0, 0, 0, 0, 5, 0, 5, 0, 0, 0, 0, 0, 0, 0, 7, 8, 8, 0, 7, 0, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [7, 0, 0, 1, 2, 2, 0, 0, 0, 1, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0], [0, 5, 0, 5, 0, 1, 0, 0, 0], [0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [ ]:
tokenizer=Tokenizer(oov_token='<UNK>')
tokenizer.fit_on_texts(training_tokens)

In [ ]:
train_sequences=tokenizer.texts_to_sequences(training_tokens)
validation_sequences=tokenizer.texts_to_sequences(validation_tokens)
test_sequences=tokenizer.texts_to_sequences(test_tokens)
print(test_sequences)

[[86, 18, 227, 442, 8273, 160, 4, 214, 6, 2764, 1060, 3], [1, 1], [1, 4, 169, 1137, 11075, 1], [227, 554, 2, 878, 5, 55, 4974, 158, 731, 24, 8, 8273, 968, 160, 79, 1641, 6, 8, 117, 435, 312, 133, 13, 96, 3], [37, 214, 1274, 55, 6116, 11877, 202, 6, 2, 80, 133, 5, 2, 117, 4, 6463, 7, 8, 2764, 789, 1060, 7, 18159, 1, 3], [214, 3024, 279, 5, 2, 133, 9, 1274, 415, 3545, 3180, 406, 2, 7731, 340, 81, 1, 1009, 1, 1, 285, 2119, 5, 8, 1, 5810, 3770, 7, 14661, 2, 1120, 73, 2, 6124, 952, 14643, 9, 110, 38, 6457, 270, 3], [1, 1, 193, 3585, 5, 2, 160, 6, 1152, 123, 4, 2563, 38, 1, 337, 1838, 735, 27, 176, 960, 2, 563, 3], [2, 149, 1835, 307, 20, 722, 6, 38, 4974, 158, 2258, 972, 16, 2, 44, 123, 3], [670, 623, 2, 4974, 140, 731, 50, 139, 390, 4, 1, 57, 6, 2, 2258, 32, 15030, 3], [50, 399, 27, 5810, 9631, 6, 2, 66, 148, 299, 484, 227, 7, 769, 27, 410, 9, 8706, 121, 85, 132, 27, 55, 779, 239, 79, 1641, 3], [1, 1, 605, 2, 899, 6, 2, 1, 340, 4, 1802, 7, 545, 8, 1, 1, 2106, 1463, 2, 4658, 953, 59, 2903, 

In [ ]:
train_padded_sequences=pad_sequences(train_sequences,padding='post',maxlen=128)
validation_padded_sequences=pad_sequences(validation_sequences,padding='post',maxlen=128)
test_padded_sequences=pad_sequences(test_sequences,padding='post',maxlen=128)

In [ ]:
token2index=tokenizer.word_index
index2token={index:token for token,index in token2index.items()}
print(token2index)
print(index2token)

{'<UNK>': 1, 'the': 2, '.': 3, ',': 4, 'of': 5, 'in': 6, 'to': 7, 'a': 8, 'and': 9, '(': 10, ')': 11, '"': 12, 'on': 13, 'said': 14, "'s": 15, 'for': 16, '1': 17, '-': 18, 'at': 19, 'was': 20, '2': 21, '0': 22, '3': 23, 'with': 24, 'that': 25, 'he': 26, 'from': 27, 'it': 28, 'by': 29, 'is': 30, ':': 31, 'as': 32, '4': 33, 'had': 34, 'his': 35, 'has': 36, 'but': 37, 'an': 38, 'not': 39, 'were': 40, 'be': 41, 'after': 42, 'have': 43, 'first': 44, 'new': 45, 'who': 46, 'will': 47, 'they': 48, '5': 49, 'two': 50, 'u.s.': 51, 'been': 52, '$': 53, '--': 54, 'their': 55, 'beat': 56, 'are': 57, '6': 58, 'which': 59, 'would': 60, 'this': 61, 'up': 62, 'its': 63, 'year': 64, 'i': 65, 'last': 66, 'percent': 67, 'out': 68, 'we': 69, 'thursday': 70, 'one': 71, 'million': 72, 'over': 73, 'government': 74, 'wednesday': 75, 'police': 76, '7': 77, 'results': 78, 'against': 79, 'second': 80, 'when': 81, '/': 82, 'also': 83, 'tuesday': 84, 'three': 85, 'soccer': 86, 'president': 87, 'no': 88, 'division':

In [ ]:
tag_names=dataset["train"].features["ner_tags"].feature.names
tag2index={tag:index for index,tag in enumerate(tag_names)}
index2tag={index:tag for tag,index in tag2index.items()}
print(tag_names)
print(tag2index)

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']
{'O': 0, 'B-PER': 1, 'I-PER': 2, 'B-ORG': 3, 'I-ORG': 4, 'B-LOC': 5, 'I-LOC': 6, 'B-MISC': 7, 'I-MISC': 8}


In [ ]:
training_padded_tags=pad_sequences(training_tags,padding='post',maxlen=128)
validation_padded_tags=pad_sequences(validation_tags,padding='post',maxlen=128)
test_padded_tags=pad_sequences(test_tags,padding='post',maxlen=128)

In [ ]:
training_categorical=tf.keras.utils.to_categorical(training_padded_tags,num_classes=len(tag2index))
validation_categorical=tf.keras.utils.to_categorical(validation_padded_tags,num_classes=len(tag2index))
test_categorical=tf.keras.utils.to_categorical(test_padded_tags,num_classes=len(tag2index))
print(training_categorical[0][0])

[0. 0. 0. 1. 0. 0. 0. 0. 0.]


In [ ]:
index2tag={index:tag for index,tag in enumerate(tag_names)}
print(index2tag)

{0: 'O', 1: 'B-PER', 2: 'I-PER', 3: 'B-ORG', 4: 'I-ORG', 5: 'B-LOC', 6: 'I-LOC', 7: 'B-MISC', 8: 'I-MISC'}


Bidirectional LSTM Model

In [ ]:
model=Sequential(
    [
        layers.Embedding(len(token2index)+1,500),
        layers.Bidirectional(LSTM(128,activation='tanh',return_sequences=True)),
        layers.Bidirectional(LSTM(128,activation='tanh',return_sequences=True)),
        layers.TimeDistributed(layers.Dense(len(tag2index),activation='softmax'))

    ]
)

In [ ]:
model.compile(loss=keras.losses.CategoricalCrossentropy(),
              optimizer=keras.optimizers.Adam(learning_rate=3e-4),
              metrics=['accuracy'])

In [ ]:
model.fit(train_padded_sequences, training_categorical, batch_size=32, epochs=10, validation_data=(validation_padded_sequences,validation_categorical))

Epoch 1/10
439/439 ━━━━━━━━━━━━━━━━━━━━ 39s 57ms/step - accuracy: 0.9661 - loss: 0.1892 - val_accuracy: 0.9806 - val_loss: 0.0649
Epoch 2/10
439/439 ━━━━━━━━━━━━━━━━━━━━ 37s 56ms/step - accuracy: 0.9872 - loss: 0.0424 - val_accuracy: 0.9907 - val_loss: 0.0336
Epoch 3/10
439/439 ━━━━━━━━━━━━━━━━━━━━ 39s 51ms/step - accuracy: 0.9955 - loss: 0.0159 - val_accuracy: 0.9934 - val_loss: 0.0250
Epoch 4/10
439/439 ━━━━━━━━━━━━━━━━━━━━ 41s 51ms/step - accuracy: 0.9979 - loss: 0.0077 - val_accuracy: 0.9939 - val_loss: 0.0227
Epoch 5/10
439/439 ━━━━━━━━━━━━━━━━━━━━ 41s 51ms/step - accuracy: 0.9989 - loss: 0.0045 - val_accuracy: 0.9937 - val_loss: 0.0246
Epoch 6/10
439/439 ━━━━━━━━━━━━━━━━━━━━ 43s 55ms/step - accuracy: 0.9992 - loss: 0.0030 - val_accuracy: 0.9940 - val_loss: 0.0260
Epoch 7/10
439/439 ━━━━━━━━━━━━━━━━━━━━ 41s 54ms/step - accuracy: 0.9995 - loss: 0.0019 - val_accuracy: 0.9938 - val_loss: 0.0309
Epoch 8/10
439/439 ━━━━━━━━━━━━━━━━━━━━ 45s 62ms/step - accuracy: 0.9996 - loss: 0.0014 - 

In [ ]:
predicted_tags_probs=model.predict(test_padded_sequences)
#print(predicted_tags_probs[0])
predicted_tags=tf.argmax(predicted_tags_probs,axis=-1).numpy()
#print(predicted_tags[0])


108/108 ━━━━━━━━━━━━━━━━━━━━ 4s 25ms/step


In [ ]:
true_tags = [[index2tag[idx] for idx in seq[:len(test_tokens[i])]] for i, seq in enumerate(test_padded_tags)]
pred_tags = [[index2tag[idx] for idx in seq[:len(test_tokens[i])]] for i, seq in enumerate(predicted_tags)]

In [ ]:
from seqeval.metrics import classification_report
print(classification_report(true_tags,pred_tags))

              precision    recall  f1-score   support

         LOC       0.74      0.71      0.72      1668
        MISC       0.70      0.62      0.66       702
         ORG       0.60      0.58      0.59      1661
         PER       0.75      0.49      0.59      1617

   micro avg       0.69      0.60      0.64      5648
   macro avg       0.70      0.60      0.64      5648
weighted avg       0.70      0.60      0.64      5648



In [ ]:
model.evaluate(test_padded_sequences,test_categorical)

108/108 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.9911 - loss: 0.0491


[0.0462476871907711, 0.9918594360351562]

#Simple RNN Model

In [ ]:
model1=Sequential(
    [
        layers.Embedding(len(token2index)+1,500),
        layers.Bidirectional(SimpleRNN(128,activation='tanh',return_sequences=True)),
        layers.Bidirectional(SimpleRNN(128,activation='tanh',return_sequences=True)),
        layers.TimeDistributed(layers.Dense(len(tag2index),activation='softmax'))

    ]
)

In [ ]:
model1.compile(loss=keras.losses.CategoricalCrossentropy(from_logits=False),
              optimizer=keras.optimizers.Adam(learning_rate=3e-4),
              metrics=['accuracy'])

In [ ]:
model1.fit(train_padded_sequences, training_categorical, batch_size=32, epochs=10, validation_data=(validation_padded_sequences,validation_categorical))

Epoch 1/10
439/439 ━━━━━━━━━━━━━━━━━━━━ 97s 148ms/step - accuracy: 0.9665 - loss: 0.1275 - val_accuracy: 0.9893 - val_loss: 0.0389
Epoch 2/10
439/439 ━━━━━━━━━━━━━━━━━━━━ 22s 50ms/step - accuracy: 0.9944 - loss: 0.0224 - val_accuracy: 0.9940 - val_loss: 0.0232
Epoch 3/10
439/439 ━━━━━━━━━━━━━━━━━━━━ 41s 49ms/step - accuracy: 0.9990 - loss: 0.0061 - val_accuracy: 0.9945 - val_loss: 0.0218
Epoch 4/10
439/439 ━━━━━━━━━━━━━━━━━━━━ 41s 49ms/step - accuracy: 0.9998 - loss: 0.0019 - val_accuracy: 0.9947 - val_loss: 0.0210
Epoch 5/10
439/439 ━━━━━━━━━━━━━━━━━━━━ 42s 51ms/step - accuracy: 0.9999 - loss: 8.5929e-04 - val_accuracy: 0.9946 - val_loss: 0.0231
Epoch 6/10
439/439 ━━━━━━━━━━━━━━━━━━━━ 43s 56ms/step - accuracy: 1.0000 - loss: 4.2892e-04 - val_accuracy: 0.9944 - val_loss: 0.0237
Epoch 7/10
439/439 ━━━━━━━━━━━━━━━━━━━━ 23s 52ms/step - accuracy: 1.0000 - loss: 2.6869e-04 - val_accuracy: 0.9945 - val_loss: 0.0253
Epoch 8/10
439/439 ━━━━━━━━━━━━━━━━━━━━ 41s 51ms/step - accuracy: 1.0000 - lo

In [ ]:
predicted_tags_probs=model1.predict(test_padded_sequences)
#print(predicted_tags_probs[0])
predicted_tags=tf.argmax(predicted_tags_probs,axis=-1).numpy()
#print(predicted_tags[0])


108/108 ━━━━━━━━━━━━━━━━━━━━ 20s 92ms/step


In [ ]:
model1.evaluate(test_padded_sequences,test_categorical)

108/108 ━━━━━━━━━━━━━━━━━━━━ 9s 81ms/step - accuracy: 0.9927 - loss: 0.0327


[0.0330963060259819, 0.9929794073104858]

Bidirectional GRU

In [ ]:
model2=Sequential(
    [
        layers.Embedding(len(token2index),300),
        layers.Bidirectional(GRU(128,activation='tanh',return_sequences=True)),
        layers.Bidirectional(GRU(128,activation='tanh',return_sequences=True)),
        layers.TimeDistributed(layers.Dense(len(tag2index),activation='softmax'))

    ]
)

In [ ]:
model2.compile(loss=keras.losses.CategoricalCrossentropy(),
              optimizer=keras.optimizers.Adam(learning_rate=3e-4),
              metrics=['accuracy'])

In [ ]:
model2.fit(train_padded_sequences, training_categorical, batch_size=32, epochs=10, validation_data=(validation_padded_sequences,validation_categorical))

Epoch 1/10
439/439 ━━━━━━━━━━━━━━━━━━━━ 34s 55ms/step - accuracy: 0.9666 - loss: 0.1870 - val_accuracy: 0.9877 - val_loss: 0.0440
Epoch 2/10
439/439 ━━━━━━━━━━━━━━━━━━━━ 41s 55ms/step - accuracy: 0.9914 - loss: 0.0289 - val_accuracy: 0.9928 - val_loss: 0.0259
Epoch 3/10
439/439 ━━━━━━━━━━━━━━━━━━━━ 41s 55ms/step - accuracy: 0.9967 - loss: 0.0121 - val_accuracy: 0.9939 - val_loss: 0.0227
Epoch 4/10
439/439 ━━━━━━━━━━━━━━━━━━━━ 39s 51ms/step - accuracy: 0.9983 - loss: 0.0065 - val_accuracy: 0.9942 - val_loss: 0.0217
Epoch 5/10
439/439 ━━━━━━━━━━━━━━━━━━━━ 41s 51ms/step - accuracy: 0.9990 - loss: 0.0038 - val_accuracy: 0.9942 - val_loss: 0.0219
Epoch 6/10
439/439 ━━━━━━━━━━━━━━━━━━━━ 40s 49ms/step - accuracy: 0.9994 - loss: 0.0024 - val_accuracy: 0.9941 - val_loss: 0.0271
Epoch 7/10
439/439 ━━━━━━━━━━━━━━━━━━━━ 42s 50ms/step - accuracy: 0.9995 - loss: 0.0018 - val_accuracy: 0.9939 - val_loss: 0.0265
Epoch 8/10
439/439 ━━━━━━━━━━━━━━━━━━━━ 41s 50ms/step - accuracy: 0.9997 - loss: 0.0011 - 

In [ ]:
model2.evaluate(test_padded_sequences,test_categorical)

108/108 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.9912 - loss: 0.0431


[0.04131776839494705, 0.9916603565216064]

In [ ]:
def make_prediction(model, index2tag, token2index, test_sentence):# tag and token
  original_tokens=test_sentence[test_sentence>0]
  original_tokens=index2tag[index] for indes in original_tokens])
  probs=model(test_sentence.reshape(1,-1))
  pridicted_tags=tf.argmax(probs[0], axis=-1).numpy()
  print(len(predicted_tags))
  predicted_tags=predicted_tags[:len(original_sentence.split())]
  predicted_tag_names=''.join([index2tag[index] for index in predicted_tags])
  return original_tokens

In [ ]:
make_prediction(model, index2tag, index2token, testing_padded_seqences[0])

NameError: name 'make_prediction' is not defined